In [333]:
import pandas as pd
import numpy as np
import scipy.sparse
from mlxtend.frequent_patterns import apriori, association_rules
# from apyori import apriori

# Tutorial: https://medium.com/analytics-vidhya/association-analysis-in-python-2b955d0180c

In [334]:
cloth_df = pd.read_csv('../dataset/stfore_train.csv')
cloth_df.head()

,external_code,retail,season,category,color,image_path,fabric,release_date,restock,0,...,2,3,4,5,6,7,8,9,10,11
0,5,36,SS17,long sleeve,grey,PE17/00005.png,acrylic,2016-11-28,0.415094,0.018868,...,0.018868,0.018868,0.037736,0.018868,0.000000,0.000000,0.037736,0.000000,0.000000,0.000000
1,2,51,SS17,long sleeve,violet,PE17/00002.png,acrylic,2016-11-28,0.320755,0.018868,...,0.018868,0.000000,0.000000,0.037736,0.000000,0.000000,0.000000,0.018868,0.018868,0.000000
2,5,10,SS17,long sleeve,grey,PE17/00005.png,acrylic,2016-11-28,0.283019,0.018868,...,0.018868,0.000000,0.018868,0.018868,0.018868,0.018868,0.018868,0.000000,0.000000,0.018868
3,9,41,SS17,culottes,yellow,PE17/00009.png,scuba crepe,2016-11-28,0.603774,0.018868,...,0.018868,0.018868,0.000000,0.000000,0.000000,0.018868,0.000000,0.018868,0.000000,0.000000
4,5,13,SS17,long sleeve,grey,PE17/00005.png,acrylic,2016-11-28,0.490566,0.018868,...,0.075472,0.000000,0.056604,0.000000,0.037736,0.018868,0.000000,0.000000,0.000000,0.000000


In [335]:
cust_df = pd.read_csv('../dataset/customer_data.csv')
cust_df.head()

,customer,retail,external_code,data,qty
0,1,0,940,2017-06-10 12:41:57,1
1,1,0,718,2017-06-10 12:41:57,1
2,1,0,825,2017-06-10 12:54:11,1
3,2,1,471,2017-08-03 11:37:27,1
4,2,1,613,2017-08-03 11:37:27,1


In [336]:
cloth_cust_df = pd.merge(cloth_df, cust_df, how='left', on=['external_code', 'retail'])
cloth_cust_df.head()

,external_code,retail,season,category,color,image_path,fabric,release_date,restock,0,...,5,6,7,8,9,10,11,customer,data,qty
0,5,36,SS17,long sleeve,grey,PE17/00005.png,acrylic,2016-11-28,0.415094,0.018868,...,0.018868,0.0,0.0,0.037736,0.0,0.0,0.0,58630.0,2017-08-12 18:43:41,1.0
1,5,36,SS17,long sleeve,grey,PE17/00005.png,acrylic,2016-11-28,0.415094,0.018868,...,0.018868,0.0,0.0,0.037736,0.0,0.0,0.0,154800.0,2017-08-12 22:31:21,1.0
2,5,36,SS17,long sleeve,grey,PE17/00005.png,acrylic,2016-11-28,0.415094,0.018868,...,0.018868,0.0,0.0,0.037736,0.0,0.0,0.0,376707.0,2016-12-15 10:07:08,1.0
3,5,36,SS17,long sleeve,grey,PE17/00005.png,acrylic,2016-11-28,0.415094,0.018868,...,0.018868,0.0,0.0,0.037736,0.0,0.0,0.0,377059.0,2016-12-06 18:22:20,1.0
4,5,36,SS17,long sleeve,grey,PE17/00005.png,acrylic,2016-11-28,0.415094,0.018868,...,0.018868,0.0,0.0,0.037736,0.0,0.0,0.0,377815.0,2016-12-15 10:30:32,1.0


In [337]:
cloth_cust_df.shape

(1472032, 24)

In [338]:
cloth_cust_df = cloth_cust_df[['external_code', 'retail', 'category', 'color', 'fabric', 'customer']]
cloth_cust_df

,external_code,retail,category,color,fabric,customer
0,5,36,long sleeve,grey,acrylic,58630.0
1,5,36,long sleeve,grey,acrylic,154800.0
2,5,36,long sleeve,grey,acrylic,376707.0
3,5,36,long sleeve,grey,acrylic,377059.0
4,5,36,long sleeve,grey,acrylic,377815.0
...,...,...,...,...,...,...
1472027,5434,41,culottes,black,cotton,170982.0
1472028,5434,41,culottes,black,cotton,219855.0
1472029,5434,41,culottes,black,cotton,224236.0
1472030,5434,41,culottes,black,cotton,224384.0


In [339]:
cloth_cust_df.dropna()

,external_code,retail,category,color,fabric,customer
0,5,36,long sleeve,grey,acrylic,58630.0
1,5,36,long sleeve,grey,acrylic,154800.0
2,5,36,long sleeve,grey,acrylic,376707.0
3,5,36,long sleeve,grey,acrylic,377059.0
4,5,36,long sleeve,grey,acrylic,377815.0
...,...,...,...,...,...,...
1472027,5434,41,culottes,black,cotton,170982.0
1472028,5434,41,culottes,black,cotton,219855.0
1472029,5434,41,culottes,black,cotton,224236.0
1472030,5434,41,culottes,black,cotton,224384.0


In [340]:
cloth_cust_df = cloth_cust_df.assign(product = cloth_cust_df.color.astype(str) + ' ' + \
    cloth_cust_df.fabric.astype(str) + ' ' + cloth_cust_df.category.astype(str))
cloth_cust_df.head()

,external_code,retail,category,color,fabric,customer,product
0,5,36,long sleeve,grey,acrylic,58630.0,grey acrylic long sleeve
1,5,36,long sleeve,grey,acrylic,154800.0,grey acrylic long sleeve
2,5,36,long sleeve,grey,acrylic,376707.0,grey acrylic long sleeve
3,5,36,long sleeve,grey,acrylic,377059.0,grey acrylic long sleeve
4,5,36,long sleeve,grey,acrylic,377815.0,grey acrylic long sleeve


In [341]:
cloth_cust_df = cloth_cust_df.drop(columns=['category', 'color', 'fabric', 'external_code', 'retail'])
cloth_cust_df.head()

,customer,product
0,58630.0,grey acrylic long sleeve
1,154800.0,grey acrylic long sleeve
2,376707.0,grey acrylic long sleeve
3,377059.0,grey acrylic long sleeve
4,377815.0,grey acrylic long sleeve


In [342]:
len(cloth_cust_df['customer'].unique())

483989

In [343]:
len(cloth_cust_df['product'].unique())

1336

In [344]:
cloth_cust_df = cloth_cust_df.dropna().sort_values('customer')

In [345]:
# cloth_cust_df.to_csv('../dataset/product.csv')

In [346]:
cloth_cust_df

,customer,product
1421472,0.0,black nice kimono dress
599194,1.0,blue linen culottes
720114,1.0,green silky satin culottes
159486,1.0,black satin cotton culottes
690379,1.0,green cotton solid color top
...,...,...
937259,667073.0,blue dark jeans culottes
1295627,667078.0,black fluid drop sleeve
1255819,667082.0,black matte jersey culottes
1117039,667084.0,green bengaline culottes


In [347]:
cloth_cust_df['product'].value_counts().sort_values(ascending=False)[:100]

black georgette doll dress       43787
black technical culottes         34788
black nice long sleeve           30283
black georgette long sleeve      21913
black scuba crepe culottes       21395
                                 ...  
black crepe trapeze dress         3210
black georgette gitana skirt      3196
black nice short sleeves          3148
blue chambree culottes            3137
violet silky satin sleeveless     3131
Name: product, Length: 100, dtype: int64

In [348]:
items = list(cloth_cust_df['product'].value_counts().sort_values(ascending=False)[:100].keys())
items = ['customer'] + items

,customer,black georgette doll dress,black technical culottes,black nice long sleeve,black georgette long sleeve,black scuba crepe culottes,black faux leather long sleeve,black georgette long dress,black bengaline culottes,black milano stitch culottes,...,black silky satin sleeveless,brown embossed drop sleeve,black devore solid color top,grey lurex solid color top,grey polyviscous culottes,black crepe trapeze dress,black georgette gitana skirt,black nice short sleeves,blue chambree culottes,violet silky satin sleeveless


In [349]:
filtered_df = cloth_cust_df[cloth_cust_df['product'].isin(items)]
filtered_df

,customer,product
159486,1.0,black satin cotton culottes
819229,1.0,black nice short cardigan
71372,2.0,white georgette sleeveless
1124994,4.0,black bengaline culottes
552465,4.0,brown cotton culottes
...,...,...
1084041,667071.0,green bengaline culottes
1084042,667072.0,green bengaline culottes
937259,667073.0,blue dark jeans culottes
1117039,667084.0,green bengaline culottes


In [350]:
len(filtered_df['product'].unique())

100

In [351]:
# cloth_cust_df = cloth_cust_df[cloth_cust_df.duplicated('customer', keep=False)]
# cloth_cust_df

In [352]:
# filter for customers who have at least 10 items and less than 10 for analysis
filtered_df = filtered_df.groupby('customer').filter(lambda x: (len(x)>10))
filtered_df

,customer,product
31259,20.0,blue dark jeans culottes
1304751,20.0,black cotton culottes
1345156,20.0,black nice long sleeve
299714,20.0,black nice long sleeve
1022106,20.0,white tulle patterned top
...,...,...
1240309,646919.0,grey georgette doll dress
1357440,646919.0,black faux leather long sleeve
1336700,646919.0,black nice short cardigan
1446354,646919.0,black georgette long sleeve


In [353]:
len(filtered_df['customer'].unique())

5178

In [354]:
filtered_df = filtered_df.groupby(['customer', 'product']).size().unstack(fill_value=0).astype(bool)
filtered_df = filtered_df.reset_index(0).reset_index(drop=True)

In [355]:
filtered_df

product,customer,black acrylic maxi,black bengaline culottes,black cloth long coat,black cloth medium coat,black cotton culottes,black cotton printed shirt,black crepe long coat,black crepe trapeze dress,black devore solid color top,...,white georgette long dress,white georgette long sleeve,white georgette short sleeves,white georgette sleeveless,white matte jersey printed shirt,white silky satin culottes,white tulle patterned top,white viscose twill short sleeves,yellow crepe trapeze dress,yellow georgette short sleeves
0,20.0,False,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,68.0,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,89.0,False,False,False,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,187.0,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,198.0,False,False,False,False,False,False,False,True,False,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5173,645556.0,False,False,False,True,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
5174,645720.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,True,False,False,False
5175,646020.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5176,646061.0,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [356]:
# for index, row in cloth_cust_df.iterrows():
#     cust = int(row['customer'])
#     product_name = row['product']

#     if cust in expanded_df['customer']:
#         expanded_df.loc[expanded_df.customer == cust, product_name] = True
#     else:
#         expanded_df.loc[len(expanded_df)] = False
#         expanded_df.iloc[-1, expanded_df.columns.get_loc('customer')] = cust
#         expanded_df.loc[expanded_df.customer == cust, product_name] = True

# expanded_df.tail()

In [357]:
filtered_df.to_csv('../dataset/ohe_products.csv', index=False)

In [358]:
expanded_df = pd.read_csv('../dataset/ohe_products.csv', index_col=False)
expanded_df.head()

,customer,black acrylic maxi,black bengaline culottes,black cloth long coat,black cloth medium coat,black cotton culottes,black cotton printed shirt,black crepe long coat,black crepe trapeze dress,black devore solid color top,...,white georgette long dress,white georgette long sleeve,white georgette short sleeves,white georgette sleeveless,white matte jersey printed shirt,white silky satin culottes,white tulle patterned top,white viscose twill short sleeves,yellow crepe trapeze dress,yellow georgette short sleeves
0,20.0,False,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,68.0,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,89.0,False,False,False,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,187.0,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,198.0,False,False,False,False,False,False,False,True,False,...,False,False,False,True,False,False,False,False,False,False


In [359]:
expanded_df = expanded_df.drop(columns=['customer'])
expanded_df

,black acrylic maxi,black bengaline culottes,black cloth long coat,black cloth medium coat,black cotton culottes,black cotton printed shirt,black crepe long coat,black crepe trapeze dress,black devore solid color top,black embossed culottes,...,white georgette long dress,white georgette long sleeve,white georgette short sleeves,white georgette sleeveless,white matte jersey printed shirt,white silky satin culottes,white tulle patterned top,white viscose twill short sleeves,yellow crepe trapeze dress,yellow georgette short sleeves
0,False,True,False,False,True,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,False,False,False,False,False,False,False,True,False,True,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5173,False,False,False,True,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
5174,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,True,False,False,False
5175,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5176,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [360]:
# mat = scipy.sparse.csr_matrix(expanded_df.values)
# sparse_df = pd.DataFrame.sparse.from_spmatrix(mat, columns=expanded_df.columns)
# sparse_df

In [361]:
common_items = apriori(expanded_df, min_support=0.01, max_len=2, use_colnames=True, verbose=1)
common_items

Processing 9900 combinations | Sampling itemset size 2


,support,itemsets
0,0.051757,(black acrylic maxi)
1,0.336423,(black bengaline culottes)
2,0.051371,(black cloth long coat)
3,0.042681,(black cloth medium coat)
4,0.314214,(black cotton culottes)
...,...,...
3046,0.019892,"(white matte jersey printed shirt, yellow geor..."
3047,0.012746,"(white tulle patterned top, white silky satin ..."
3048,0.017767,"(yellow georgette short sleeves, white silky s..."
3049,0.017574,"(white tulle patterned top, yellow georgette s..."


In [362]:
# common_items.to_csv('../dataset/common_items.csv')

In [363]:
rules = association_rules(common_items, metric="confidence", min_threshold=0.3)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(black acrylic maxi),(black bengaline culottes),0.051757,0.336423,0.018926,0.365672,1.086939,0.001514,1.046109
1,(black acrylic maxi),(black cotton culottes),0.051757,0.314214,0.019892,0.384328,1.223142,0.003629,1.113883
2,(black acrylic maxi),(black faux leather long sleeve),0.051757,0.277520,0.015643,0.302239,1.089069,0.001279,1.035425
3,(black acrylic maxi),(black georgette doll dress),0.051757,0.352646,0.020857,0.402985,1.142747,0.002605,1.084318
4,(black acrylic maxi),(black georgette long sleeve),0.051757,0.334492,0.018540,0.358209,1.070904,0.001228,1.036954
...,...,...,...,...,...,...,...,...,...
897,(red georgette sleeveless),(violet georgette short sleeves),0.088644,0.226728,0.028003,0.315904,1.393315,0.007905,1.130356
898,(red georgette sleeveless),(white georgette sleeveless),0.088644,0.211278,0.026844,0.302832,1.433332,0.008116,1.131322
899,(white georgette short sleeves),(violet georgette short sleeves),0.215913,0.226728,0.067207,0.311270,1.372876,0.018254,1.122750
900,(white silky satin culottes),(violet georgette short sleeves),0.099846,0.226728,0.038046,0.381044,1.680620,0.015408,1.249317


In [364]:
rules.sort_values('support', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
632,(black technical culottes),(black nice long sleeve),0.414832,0.455002,0.194477,0.468808,1.030343,0.005727,1.025991
631,(black nice long sleeve),(black technical culottes),0.455002,0.414832,0.194477,0.427419,1.030343,0.005727,1.021984
623,(black nice long sleeve),(black scuba crepe culottes),0.455002,0.335844,0.167053,0.367148,1.093209,0.014243,1.049465
624,(black scuba crepe culottes),(black nice long sleeve),0.335844,0.455002,0.167053,0.497412,1.093209,0.014243,1.084384
154,(black nice long sleeve),(black cotton culottes),0.455002,0.314214,0.159714,0.351019,1.117133,0.016746,1.056711
...,...,...,...,...,...,...,...,...,...
126,(black cloth medium coat),(blue dark jeans culottes),0.042681,0.305716,0.013326,0.312217,1.021264,0.000277,1.009452
326,(red georgette long dress),(black faux leather long sleeve),0.037080,0.277520,0.013132,0.354167,1.276183,0.002842,1.118678
198,(red georgette long dress),(black cotton culottes),0.037080,0.314214,0.012553,0.338542,1.077424,0.000902,1.036779
89,(red georgette long dress),(black bengaline culottes),0.037080,0.336423,0.012360,0.333333,0.990815,-0.000115,0.995365


In [365]:
rules['antecedents'][0]

frozenset({'black acrylic maxi'})

In [366]:
rules["antecedents"].apply(lambda x: list(x)[0]).astype("unicode")

0                 black acrylic maxi
1                 black acrylic maxi
2                 black acrylic maxi
3                 black acrylic maxi
4                 black acrylic maxi
                   ...              
897         red georgette sleeveless
898         red georgette sleeveless
899    white georgette short sleeves
900       white silky satin culottes
901       white silky satin culottes
Name: antecedents, Length: 902, dtype: object

In [367]:
cols = ['antecedents','consequents']
rules[cols] = rules[cols].astype('string')

rules['antecedents'] = rules['antecedents'].str.removeprefix("frozenset({'")
rules['antecedents'] = rules['antecedents'].str.removesuffix("'})")

rules['consequents'] = rules['consequents'].str.removeprefix("frozenset({'")
rules['consequents'] = rules['consequents'].str.removesuffix("'})")

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,black acrylic maxi,black bengaline culottes,0.051757,0.336423,0.018926,0.365672,1.086939,0.001514,1.046109
1,black acrylic maxi,black cotton culottes,0.051757,0.314214,0.019892,0.384328,1.223142,0.003629,1.113883
2,black acrylic maxi,black faux leather long sleeve,0.051757,0.277520,0.015643,0.302239,1.089069,0.001279,1.035425
3,black acrylic maxi,black georgette doll dress,0.051757,0.352646,0.020857,0.402985,1.142747,0.002605,1.084318
4,black acrylic maxi,black georgette long sleeve,0.051757,0.334492,0.018540,0.358209,1.070904,0.001228,1.036954
...,...,...,...,...,...,...,...,...,...
897,red georgette sleeveless,violet georgette short sleeves,0.088644,0.226728,0.028003,0.315904,1.393315,0.007905,1.130356
898,red georgette sleeveless,white georgette sleeveless,0.088644,0.211278,0.026844,0.302832,1.433332,0.008116,1.131322
899,white georgette short sleeves,violet georgette short sleeves,0.215913,0.226728,0.067207,0.311270,1.372876,0.018254,1.122750
900,white silky satin culottes,violet georgette short sleeves,0.099846,0.226728,0.038046,0.381044,1.680620,0.015408,1.249317


In [368]:
rules.loc[rules['antecedents'] == 'black acrylic maxi']['consequents'].tolist()

['black bengaline culottes',
 'black cotton culottes',
 'black faux leather long sleeve',
 'black georgette doll dress',
 'black georgette long sleeve',
 'black matte jersey solid color top',
 'black nice long sleeve',
 'black scuba crepe culottes',
 'black technical culottes',
 'blue dark jeans culottes']

In [400]:
aritzia = pd.read_csv('../dataset/aritzia_test.csv', index_col=False)
aritzia

,customer,product,aritzia
0,1.0,black satin cotton culottes,The Melina™ Pant
1,1.0,black nice short cardigan,Refined Cargo Pant
2,2.0,white georgette sleeveless,Izzy Cardigan
3,4.0,black bengaline culottes,Justin Jacket
4,4.0,brown cotton culottes,Unwind Cardigan
...,...,...,...
801463,667071.0,green bengaline culottes,Weekend T-Shirt
801464,667072.0,green bengaline culottes,Weekend T-Shirt
801465,667073.0,blue dark jeans culottes,Society Sweater
801466,667084.0,green bengaline culottes,Weekend T-Shirt


In [401]:
aritzia = aritzia.drop(columns='product')

In [402]:
aritzia = aritzia.groupby('customer').filter(lambda x: (len(x)>10))
aritzia

,customer,aritzia
18,20.0,Society Sweater
19,20.0,TnaCHILL™ Atmosphere Hi-Rise Legging
20,20.0,Divinity Jumpsuit
21,20.0,Divinity Jumpsuit
22,20.0,Ronnie Sweater
...,...,...
794504,646919.0,Ribbed Waist T-Shirt
794505,646919.0,The Ganna™ Shirt Jacket
794506,646919.0,Refined Cargo Pant
794507,646919.0,Cozy Fleece Boyfriend Hoodie


In [398]:
aritzia = aritzia.groupby(['customer', 'aritzia']).size().unstack(fill_value=0).astype(bool)
aritzia = aritzia.reset_index(0).reset_index(drop=True)

In [399]:
aritzia

aritzia,customer,Agency Blazer,Alanya Pant,Alchemy Dress,Alftan Luxe Cashmere Cardigan,Ascendant Pant,Aster Free Lounge Pant,Avery Overall,Bigfoot Polar Zip Hoodie,Brennan Pant,...,The Super Puff™ Long,The Super Puff™ Mid,The Super Puff™ Shorty,Thermal Cowlneck,TnaBUTTER™ Rhythm Jumpsuit,TnaCHILL™ Atmosphere Hi-Rise Legging,Troop Cargo Pant,Ty Cardigan,Unwind Cardigan,Weekend T-Shirt
0,20.0,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,68.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,89.0,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
3,187.0,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,198.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5173,645556.0,False,False,False,False,False,False,False,False,False,...,False,False,False,True,True,True,True,False,False,False
5174,645720.0,False,False,False,True,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
5175,646020.0,True,False,False,True,True,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False
5176,646061.0,False,False,False,False,False,False,True,False,True,...,False,False,False,False,False,True,False,True,False,False


In [392]:
aritzia = aritzia.drop(columns=['customer'])
aritzia

aritzia,Agency Blazer,Alanya Pant,Alchemy Dress,Alftan Luxe Cashmere Cardigan,Ascendant Pant,Aster Free Lounge Pant,Avery Overall,Bigfoot Polar Zip Hoodie,Brennan Pant,Carter Blouse,...,The Super Puff™ Long,The Super Puff™ Mid,The Super Puff™ Shorty,Thermal Cowlneck,TnaBUTTER™ Rhythm Jumpsuit,TnaCHILL™ Atmosphere Hi-Rise Legging,Troop Cargo Pant,Ty Cardigan,Unwind Cardigan,Weekend T-Shirt
0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5173,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,True,True,True,False,False,False
5174,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
5175,True,False,False,True,True,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
5176,False,False,False,False,False,False,True,False,True,False,...,False,False,False,False,False,True,False,True,False,False


In [393]:
a_common_items = apriori(aritzia, min_support=0.01, max_len=2, use_colnames=True, verbose=1)
a_common_items

Processing 8372 combinations | Sampling itemset size 2


,support,itemsets
0,0.128814,(Agency Blazer)
1,0.085554,(Alanya Pant)
2,0.226728,(Alchemy Dress)
3,0.252993,(Alftan Luxe Cashmere Cardigan)
4,0.093086,(Ascendant Pant)
...,...,...
2793,0.033797,"(Troop Cargo Pant, Unwind Cardigan)"
2794,0.057937,"(Weekend T-Shirt, Troop Cargo Pant)"
2795,0.011974,"(Ty Cardigan, Unwind Cardigan)"
2796,0.023561,"(Weekend T-Shirt, Ty Cardigan)"


In [394]:
ar_rules = association_rules(a_common_items, metric="confidence", min_threshold=0.2)
ar_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Agency Blazer),(Alchemy Dress),0.128814,0.226728,0.031672,0.245877,1.084456,0.002467,1.025392
1,(Agency Blazer),(Alftan Luxe Cashmere Cardigan),0.128814,0.252993,0.031093,0.241379,0.954093,-0.001496,0.984690
2,(Agency Blazer),(Avery Overall),0.128814,0.335844,0.049440,0.383808,1.142817,0.006178,1.077840
3,(Agency Blazer),(Contour Crew Longsleeve Bodysuit),0.128814,0.215913,0.032059,0.248876,1.152663,0.004246,1.043884
4,(Agency Blazer),(Cozy Fleece Boyfriend Hoodie),0.128814,0.334492,0.040363,0.313343,0.936774,-0.002724,0.969200
...,...,...,...,...,...,...,...,...,...
1963,(Weekend T-Shirt),(TnaCHILL™ Atmosphere Hi-Rise Legging),0.179606,0.314214,0.053302,0.296774,0.944497,-0.003132,0.975200
1964,(Ty Cardigan),(Troop Cargo Pant),0.135574,0.318463,0.042681,0.314815,0.988545,-0.000495,0.994676
1965,(Unwind Cardigan),(Troop Cargo Pant),0.092314,0.318463,0.033797,0.366109,1.149613,0.004398,1.075164
1966,(Weekend T-Shirt),(Troop Cargo Pant),0.179606,0.318463,0.057937,0.322581,1.012931,0.000740,1.006079


In [404]:
ar_rules = ar_rules.sort_values('support', ascending=False)

In [405]:
cols = ['antecedents','consequents']
ar_rules[cols] = ar_rules[cols].astype('string')

ar_rules['antecedents'] = ar_rules['antecedents'].str.removeprefix("frozenset({'")
ar_rules['antecedents'] = ar_rules['antecedents'].str.removesuffix("'})")

ar_rules['consequents'] = ar_rules['consequents'].str.removeprefix("frozenset({'")
ar_rules['consequents'] = ar_rules['consequents'].str.removesuffix("'})")

ar_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
762,Divinity Jumpsuit,Cozy Fleece Perfect Zip Hoodie,0.455002,0.414832,0.194477,0.427419,1.030343,0.005727,1.021984
763,Cozy Fleece Perfect Zip Hoodie,Divinity Jumpsuit,0.414832,0.455002,0.194477,0.468808,1.030343,0.005727,1.025991
974,The Powder Parka™,Divinity Jumpsuit,0.398803,0.455002,0.177289,0.444552,0.977033,-0.004167,0.981187
973,Divinity Jumpsuit,The Powder Parka™,0.455002,0.398803,0.177289,0.389643,0.977033,-0.004167,0.984994
312,Divinity Jumpsuit,Avery Overall,0.455002,0.335844,0.167053,0.367148,1.093209,0.014243,1.049465
...,...,...,...,...,...,...,...,...,...
1513,The Stedman Coat,Runaway Blouse,0.051757,0.118385,0.010429,0.201493,1.702004,0.004301,1.104078
555,The Cocoon Long Coat,Contour Crew Longsleeve Bodysuit,0.050019,0.215913,0.010236,0.204633,0.947756,-0.000564,0.985818
1093,Harper Sweater,Ty Cardigan,0.050019,0.135574,0.010042,0.200772,1.480909,0.003261,1.081577
1090,Harper Sweater,The Super Puff™ Long,0.050019,0.178834,0.010042,0.200772,1.122677,0.001097,1.027450


In [409]:
ar_rules_items = ar_rules.drop(columns=['antecedent support', 'consequent support', 'support', 'confidence', 'lift', 'leverage', 'conviction'])
ar_rules_items

,antecedents,consequents
762,Divinity Jumpsuit,Cozy Fleece Perfect Zip Hoodie
763,Cozy Fleece Perfect Zip Hoodie,Divinity Jumpsuit
974,The Powder Parka™,Divinity Jumpsuit
973,Divinity Jumpsuit,The Powder Parka™
312,Divinity Jumpsuit,Avery Overall
...,...,...
1513,The Stedman Coat,Runaway Blouse
555,The Cocoon Long Coat,Contour Crew Longsleeve Bodysuit
1093,Harper Sweater,Ty Cardigan
1090,Harper Sweater,The Super Puff™ Long


In [410]:
ar_rules_items.to_csv('../dataset/aritzia_rules.csv')